<a href="https://colab.research.google.com/github/Pinskerella/macro-economic-volatility-modeling/blob/main/GDP_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import duckdb

# 1. Connect to an in-memory database (Zero setup)
con = duckdb.connect()

# 2. Use a direct URL to a clean version of the data
# This bypasses the 'File Not Found' and 'Header' issues
url = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"

# 3. The Math: Calculate Growth & Volatility in one go
df = con.execute(f"""
    WITH raw_data AS (
        SELECT "Country Name" as country, Value as gdp, Year as year
        FROM read_csv_auto('{url}')
    ),
    growth_calc AS (
        SELECT *,
               (gdp - LAG(gdp) OVER (PARTITION BY country ORDER BY year)) /
               NULLIF(LAG(gdp) OVER (PARTITION BY country ORDER BY year), 0) as growth_rate
        FROM raw_data
    )
    SELECT country,
           ROUND(AVG(growth_rate) * 100, 2) as avg_growth_pct,
           ROUND(STDDEV(growth_rate), 4) as volatility_score
    FROM growth_calc
    GROUP BY country
    HAVING COUNT(*) > 20
    ORDER BY volatility_score DESC
    LIMIT 10;
""").df()

# 4. Display the results
print("MODULE 1: TOP 10 MOST VOLATILE ECONOMIES")
print("========================================")
print(df)

MODULE 1: TOP 10 MOST VOLATILE ECONOMIES
                country  avg_growth_pct  volatility_score
0               Georgia      1483076.21        88984.1979
1     Equatorial Guinea           20.33            0.5895
2      Congo, Dem. Rep.           14.20            0.5716
3                  Iraq           25.43            0.5704
4            Azerbaijan           18.93            0.5419
5  United Arab Emirates           16.86            0.3711
6                  Oman           16.89            0.3517
7                 Qatar           16.72            0.3226
8               Nigeria           11.49            0.3123
9                Angola           13.27            0.2944


In [ ]:
import duckdb
import plotly.express as px

# 1. Connect and Load Clean Data
con = duckdb.connect()
url = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"

# 2. SQL: Calculate the Math (Growth and Volatility)
df = con.execute(f"""
    WITH growth_calc AS (
        SELECT "Country Name" as country, Value as gdp, Year as year,
               (Value - LAG(Value) OVER (PARTITION BY "Country Name" ORDER BY Year)) /
               NULLIF(LAG(Value) OVER (PARTITION BY "Country Name" ORDER BY Year), 0) as growth_rate
        FROM read_csv_auto('{url}')
    )
    SELECT country,
           AVG(growth_rate) * 100 as avg_growth,
           STDDEV(growth_rate) as volatility
    FROM growth_calc
    GROUP BY country
    HAVING COUNT(*) > 30
    ORDER BY volatility DESC
    LIMIT 15;
""").df()

# 3. Visual: Create the Volatility Chart
fig = px.scatter(df, x="avg_growth", y="volatility", text="country",
                 title="Macro-Economic Volatility vs. Growth",
                 labels={"avg_growth": "Average Annual Growth (%)", "volatility": "Volatility (Std Dev)"},
                 size="volatility", color="volatility", template="plotly_white")

fig.update_traces(textposition='top center')
fig.show()

"Excluded Georgia as a statistical outlier to maintain chart legibility and focus on broader regional trends."

In [ ]:
# Updated SQL for Module 2
df = con.execute(f"""
    WITH growth_calc AS (
        SELECT "Country Name" as country, Value as gdp, Year as year,
               (Value - LAG(Value) OVER (PARTITION by "Country Name" ORDER BY Year)) /
               NULLIF(LAG(Value) OVER (PARTITION by "Country Name" ORDER BY Year), 0) as growth_rate
        FROM read_csv_auto('{url}')
    )
    SELECT country,
           AVG(growth_rate) * 100 as avg_growth,
           STDDEV(growth_rate) as volatility
    FROM growth_calc
    WHERE country != 'Georgia'  -- This line removes the outlier
    GROUP BY country
    HAVING COUNT(*) > 30
    ORDER BY volatility DESC
    LIMIT 15;
""").df()

# Re-run the px.scatter code below this...
# 3. Visual: Create the Volatility Chart
fig = px.scatter(df, x="avg_growth", y="volatility", text="country",
                 title="Macro-Economic Volatility vs. Growth",
                 labels={"avg_growth": "Average Annual Growth (%)", "volatility": "Volatility (Std Dev)"},
                 size="volatility", color="volatility", template="plotly_white")

fig.update_traces(textposition='top center')
fig.show()

In [ ]:
import duckdb
import plotly.express as px

# 1. Connect and Load
con = duckdb.connect()
url = "https://raw.githubusercontent.com/datasets/gdp/master/data/gdp.csv"

# 2. SQL: Calculate Math & Filter Outliers
# We exclude 'Georgia' to fix the scale issue
df = con.execute(f"""
    WITH growth_calc AS (
        SELECT "Country Name" as country, Value as gdp, Year as year,
               (Value - LAG(Value) OVER (PARTITION BY "Country Name" ORDER BY Year)) /
               NULLIF(LAG(Value) OVER (PARTITION BY "Country Name" ORDER BY Year), 0) as growth_rate
        FROM read_csv_auto('{url}')
    )
    SELECT country,
           AVG(growth_rate) * 100 as avg_growth,
           STDDEV(growth_rate) as volatility
    FROM growth_calc
    WHERE country NOT IN ('Georgia', 'Equatorial Guinea') -- Removing extreme outliers
    GROUP BY country
    HAVING COUNT(*) > 30
    ORDER BY volatility DESC
    LIMIT 15;
""").df()

# 3. Visual: Scatter Plot with Trend Line
fig = px.scatter(df, x="avg_growth", y="volatility", text="country",
                 title="Macro-Economic Volatility vs. Growth (Outliers Removed)",
                 labels={"avg_growth": "Average Annual Growth (%)", "volatility": "Volatility (Risk Score)"},
                 trendline="ols", # Adds the mathematical trend line
                 size="volatility", color="avg_growth",
                 color_continuous_scale=px.colors.sequential.Viridis,
                 template="plotly_white")

fig.update_traces(textposition='top center')
fig.show()

In [6]:
# Choose your language: 'en', 'es', 'fr', or 'de'
target_lang = 'en'

summaries = {
    'en': f"This analysis identifies the top {len(df)} countries with the highest GDP volatility. The trend line (OLS) indicates the risk-reward ratio for emerging markets. High-volatility outliers like Georgia were excluded to ensure statistical relevance.",
    'es': f"Este análisis identifica los {len(df)} países con la mayor volatilidad del PIB. La línea de tendencia (OLS) indica la relación riesgo-recompensa. Se excluyeron valores atípicos como Georgia para garantizar la relevancia estadística.",
    'fr': f"Cette analyse identifie les {len(df)} pays présentant la plus forte volatilité du PIB. La ligne de tendance (OLS) indique le rapport risque-rendement. Les valeurs aberrantes comme la Géorgie ont été exclues pour garantir la pertinence statistique.",
    'de': f"Diese Analyse identifiziert die {len(df)} Länder mit der höchsten BIP-Volatilität. Die Trendlinie (OLS) zeigt das Risiko-Rendite-Verhältnis. Ausreißer wie Georgien wurden ausgeschlossen, um die statistische Relevanz zu gewährleisten."
}

print("--- PORTFOLIO SUMMARY ---")
print(summaries.get(target_lang, summaries['en']))

--- PORTFOLIO SUMMARY ---
This analysis identifies the top 15 countries with the highest GDP volatility. The trend line (OLS) indicates the risk-reward ratio for emerging markets. High-volatility outliers like Georgia were excluded to ensure statistical relevance.
